# 2D Elliptic equations

In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
from time import process_time

## Solver

In [ ]:
def solve_poisson(x_lims, y_lims, n, f, g):
    
    # grids
    x_l, x_u = x_lims
    y_l, y_u = y_lims
    n_x, n_y = n
    x = np.linspace(x_l, x_u, n_x)
    y = np.linspace(y_l, y_u, n_y)
    X, Y = np.meshgrid(x, y)
    
    dx = x[1] - x[0]
    dy = y[1] - y[0]
    
    # number of unknowns
    n_unk = n_x*n_y

    # initialize right-hand side
    b = np.zeros(n_unk)

    # set up the Poisson system
    I, J, data = [], [], []

    # "lazy way" do do the loop
    for i in range(n_y):
        yi = y[i]
        for j in range(n_x):
            xj = x[j]

            k = i*n_x + j

            # check if this is a boundary point
            if i == 0 or i == n_y-1 or j == 0 or j == n_x-1:
                I.append(k)
                J.append(k)
                data.append(1)
                b[k] = g(xj, yi)
            else:
                I.extend([k, k, k, k, k])
                J.extend([k-n_x, k-1, k, k+1, k+n_x])
                data.extend([1/dy**2, 1/dx**2, -2/dx**2 -2/dy**2, 1/dx**2, 1/dy**2])
                b[k] = f(xj, yi)

    # build the sparse matrix
    A = sp.csr_matrix((data, (I,J)), (n_unk, n_unk))
    
    # solve the system
    u = sla.spsolve(A, b)
    
    # convert to array of solution values
    u = u.reshape(n_y, n_x)
    
    return u

## Convergence analysis

In [ ]:
def convergence_test():
    
    # solution coefficients
    sol_a = 2.8
    sol_b = -1.1
    sol_c = 1.8
    sol_d = 2.2

    def sol(x, y):
        "True solution"
        return np.sin(sol_a*x + sol_b)*np.cos(sol_c*y + sol_d)

    def f(x, y):
        "Right-hand side function"
        return -(sol_a**2 + sol_c**2)*sol(x, y)

    # grid coefficients
    x_l, x_u = -1.8, 2.3
    y_l, y_u = -1.7, 2.4
    
    # previous error
    prev_err = None
    prev_h   = None
    
    print()
    print('|     h     |   error   | order |')
    print('+-----------+-----------+-------+')
    
    for test in range(6):
        n_x = 2**test * 10
        n_y = n_x + 3

        # 1D grids
        x = np.linspace(x_l, x_u, n_x)
        y = np.linspace(y_l, y_u, n_y)
        
        # grid "size"
        h = min(x[1]-x[0], y[1]-y[0])

        # 2D grids
        X, Y = np.meshgrid(x, y)

        # solve the Poisson system
        u = solve_poisson((x_l, x_u), (y_l, y_u), (n_x, n_y), f, sol)
        err = np.max(np.abs(u - sol(X, Y)))
        
        print('| {:.3e} | {:.3e} |'.format(h, err), end='')
        if prev_err is None:
            print('  N/A  |')
            
        else:
            order = np.log(prev_err/err)/np.log(prev_h/h)
            print(' {:.3f} |'.format(order))
        prev_err = err
        prev_h = h
        
        
    print()
    
convergence_test()

## Error as a function of number of unknowns

In [ ]:
def convergence_test_2():
    
    # solution coefficients
    sol_a = 2.8
    sol_b = -1.1
    sol_c = 1.8
    sol_d = 2.2

    def sol(x, y):
        "True solution"
        return np.sin(sol_a*x + sol_b)*np.cos(sol_c*y + sol_d)

    def f(x, y):
        "Right-hand side function"
        return -(sol_a**2 + sol_c**2)*sol(x, y)

    # grid coefficients
    x_l, x_u = -1.8, 2.3
    y_l, y_u = -1.7, 2.4
    
    # previous error
    prev_err = None
    prev_n_unk = None
    
    print()
    print('|   n_unk   |   error   | order |')
    print('+-----------+-----------+-------+')
    
    for test in range(6):
        n_x = 2**test * 10
        n_y = n_x + 3
        
        n_unk = n_x*n_y

        # 1D grids
        x = np.linspace(x_l, x_u, n_x)
        y = np.linspace(y_l, y_u, n_y)
        
        # grid "size"
        h = min(x[1]-x[0], y[1]-y[0])

        # 2D grids
        X, Y = np.meshgrid(x, y)

        # solve the Poisson system
        u = solve_poisson((x_l, x_u), (y_l, y_u), (n_x, n_y), f, sol)
        err = np.max(np.abs(u - sol(X, Y)))
        
        print('| {:9d} | {:.3e} |'.format(n_unk, err), end='')
        if prev_err is None:
            print('  N/A  |')
            
        else:
            order = np.log(prev_err/err)/np.log(n_unk/prev_n_unk)
            print(' {:.3f} |'.format(order))
        prev_err = err
        prev_n_unk = n_unk
        
        
    print()
    
convergence_test_2()

## Timing

In [ ]:
def timing_test():
    
    # solution coefficients
    sol_a = 2.8
    sol_b = -1.1
    sol_c = 1.8
    sol_d = 2.2

    def sol(x, y):
        "True solution"
        return np.sin(sol_a*x + sol_b)*np.cos(sol_c*y + sol_d)

    def f(x, y):
        "Right-hand side function"
        return -(sol_a**2 + sol_c**2)*sol(x, y)

    # grid coefficients
    x_l, x_u = -1.8, 2.3
    y_l, y_u = -1.7, 2.4
    
    # previous error
    prev_time = None
    prev_n_unk = None
    
    print()
    print('|   n_unk   |    time   | order |')
    print('+-----------+-----------+-------+')
    
    for test in range(6):
        n_x = 2**test * 10
        n_y = n_x + 3
        
        n_unk = n_x*n_y

        # 1D grids
        x = np.linspace(x_l, x_u, n_x)
        y = np.linspace(y_l, y_u, n_y)
        
        # grid "size"
        h = min(x[1]-x[0], y[1]-y[0])

        # 2D grids
        X, Y = np.meshgrid(x, y)

        # solve the Poisson system (average over several solves)
        start = process_time()
        for k in range(5):
            u = solve_poisson((x_l, x_u), (y_l, y_u), (n_x, n_y), f, sol)
        time = (process_time() - start)/5
        
        print('| {:9d} | {:.3e} |'.format(n_unk, time), end='')
        if prev_time is None:
            print('  N/A  |')
            
        else:
            order = np.log(time/prev_time)/np.log(n_unk/prev_n_unk)
            print(' {:.3f} |'.format(order))
        prev_time = time
        prev_n_unk = n_unk
        
        
    print()
    
timing_test()

## Examples

In [ ]:
x_l, x_u = 0, 4
y_l, y_u = 0, 1
n_x, n_y = 401, 101

x = np.linspace(x_l, x_u, n_x)
y = np.linspace(y_l, y_u, n_y)

def f(x, y):
    return np.zeros(x.shape)
    #return -np.exp(-(x-1)**4-(y-0.5)**4)

def sc_g(x, y):
    if (x <= 1e-12 and abs(y-0.5) < 0.25) or (x >= 2 and y <= 0.5):
        return 1
    else:
        return 0

g = np.vectorize(sc_g)
    
u = solve_poisson((x_l, x_u), (y_l, y_u), (n_x, n_y), f, g)
plt.figure(figsize=(12,3))
plt.pcolor( x, y, u, cmap='jet');

## Jacobi iteration

### Matrix-free (pretty slow in Python)

In [ ]:
x_l, x_u = 0, 4
y_l, y_u = 0, 1
n_x, n_y = 101, 51

x = np.linspace(x_l, x_u, n_x)
y = np.linspace(y_l, y_u, n_y)

dx, dy = x[1]-x[0], y[1]-y[0]

X, Y = np.meshgrid(x, y)

def f(x, y):
    return np.zeros(x.shape)
    #return -np.exp(-(x-1)**4-(y-0.5)**4)

def sc_g(x, y):
    if (x <= 1e-12 and abs(y-0.5) < 0.25) or (x >= 2 and y <= 0.5):
        return 1
    else:
        return 0

g = np.vectorize(sc_g)

# initial u
u = np.zeros((n_y, n_x))

# fill in boundary conditions
u[0, :] = g(x, y[0])
u[-1, :] = g(x, y[-1])
u[:, 0] = g(x[0], y)
u[:, -1] = g(x[-1], y)

# scaled right-hand side
sf = -(dx**2*dy**2)/(2*(dx**2 + dy**2))*f(X, Y)

# coefficients for update
cx = dy**2/(2*(dx**2 + dy**2))
cy = dx**2/(2*(dx**2 + dy**2))

# iterate several times
n_iter = 100
for it in range(n_iter):
    
    u_old = u.copy()
    
    # loop over interior points
    for i in range(1, n_y-1):
        for j in range(1, n_x-1):
            
            # update
            u[i, j] = cx*(u_old[i, j-1] + u_old[i, j+1]) + cy*(u_old[i-1, j] + u_old[i+1, j]) + sf[i, j]
            
plt.figure(figsize=(12,3))
plt.pcolor( x, y, u, cmap='jet');

### Using sparse matrices is faster in an interpreted language

In [ ]:
def jacobi_iterations(x_lims, y_lims, n, f, g, n_iter):
    
    # grids
    x_l, x_u = x_lims
    y_l, y_u = y_lims
    n_x, n_y = n
    x = np.linspace(x_l, x_u, n_x)
    y = np.linspace(y_l, y_u, n_y)
    X, Y = np.meshgrid(x, y)
    
    dx = x[1] - x[0]
    dy = y[1] - y[0]
    
    # number of unknowns
    n_unk = n_x*n_y

    # initialize right-hand side
    b = np.zeros(n_unk)

    # data structure for off-diagonal entries
    I, J, data = [], [], []
    
    # vector for inverse of diagonal entries
    di = np.zeros(n_unk)

    # "lazy way" do do the loop
    for i in range(n_y):
        yi = y[i]
        for j in range(n_x):
            xj = x[j]

            k = i*n_x + j

            # check if this is a boundary point
            if i == 0 or i == n_y-1 or j == 0 or j == n_x-1:
                di[k] = 1
                b[k] = g(xj, yi)
            else:
                I.extend([k, k, k, k])
                J.extend([k-n_x, k-1, k+1, k+n_x])
                data.extend([1/dy**2, 1/dx**2, 1/dx**2, 1/dy**2])
                di[k] = 1/(-2/dx**2 -2/dy**2)
                b[k] = f(xj, yi)

    # build the sparse matrix
    R = sp.csr_matrix((data, (I,J)), (n_unk, n_unk))

    # initialize u
    u = np.zeros(n_unk)
    
    # iterate
    for it in range(n_iter):
        u = di*(b - R*u)
    
    # convert to array of solution values
    u = u.reshape(n_y, n_x)
    
    return u

In [ ]:
x_l, x_u = 0, 4
y_l, y_u = 0, 1
n_x, n_y = 401, 101

x = np.linspace(x_l, x_u, n_x)
y = np.linspace(y_l, y_u, n_y)

u = jacobi_iterations((x_l, x_u), (y_l, y_u), (n_x, n_y), f, g, 5000)

plt.figure(figsize=(12,3))
plt.pcolor( x, y, u, cmap='jet');